In [1]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.

In [2]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
from guidance import gen
from evaluate import load
import csv
import nltk.translate.bleu_score as bleu
from rouge import Rouge
from transformers import BertTokenizer, BertModel
import torch
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import warnings
from typing import Any, Dict, Tuple
import re

warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Load Dataset

- id: unique identifier of the example
- source: original model input for paraphrase generation
- hyp1: first alternative paraphrase of the source
- hyp2: second alternative paraphrase of the source
- label: hyp1 or hyp2, based on which of those has been annotated as hallucination
- type: hallucination category assigned.
  - Possible values: addition, named-entity, number, conversion, date, tense, negation, gender, pronoun, antonym, natural

## Trial dataset

This is a small list of examples, provided to help the participants get familiar with the task. Each example contains the following fields:
- id
- source
- hyp1
- hyp2
- label
- type

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [5]:
from datasets import load_dataset

#load the trial data for both English and Swedish
trial_ds = load_dataset("Eloquent/HalluciGen-PG", name="trial")

#load the trial data only for Swedish
trial_ds_sv = load_dataset("Eloquent/HalluciGen-PG", name="trial", split="trial_swedish")

print(trial_ds.keys())

trial_df_en = trial_ds['trial_english'].to_pandas()
trial_df_se= trial_ds['trial_swedish'].to_pandas()

print('English:')

display(trial_df_en.head())

print('Swedish')
display(trial_df_se.head())

Generating trial_swedish split:   0%|          | 0/20 [00:00<?, ? examples/s]

Generating trial_english split:   0%|          | 0/16 [00:00<?, ? examples/s]

dict_keys(['trial_swedish', 'trial_english'])
English:


,id,source,type,hyp1,hyp2,label
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2


Swedish


,id,source,type,hyp1,hyp2,label
0,0,"Men intäkterna från mjukvarulicenser, ett mått...",number,"Intäkter från programvarulicenser, en metrik s...","Intäkter från programvarulicenser, en metrik s...",hyp2
1,1,Hongkong-universitet samarbetar med universite...,natural,University of Hong Kong samarbetar med olika k...,University of Hong Kong samarbetar med olika k...,hyp1
2,2,Län med befolkningsminskning kommer att vara V...,named entity,"Vermillion, Posey och Madison är län som komme...","Vermillion, Posey och Marion är län som kommer...",hyp2
3,3,Google presenterar en prototyp för en självkör...,addition,En prototyp för en självkörande bil presentera...,Enligt ett blogginlägg från Google har företag...,hyp2
4,4,Lagförslaget säger att en kvinna som genomgår ...,negation,Förslaget stadgar att en kvinna som genomgår e...,Förslaget stadgar att en kvinna som genomgår e...,hyp1


In [6]:
print(trial_df_en['type'].unique())


['antonym' 'negation' 'named entity' 'natural' 'addition' 'gender'
 'number' 'pronoun' 'conversion' 'date']


In [7]:
print('English:')

display(trial_df_en.head())

English:


,id,source,type,hyp1,hyp2,label
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2


In [8]:
print('English')
row = trial_df_en.iloc[6]  # Selecting the first row
print("Source:", row['source'])
print("hyp1:", row['hyp1'])
print("hyp2:", row['hyp2'])
print("label:",row['label'] )
print("type:",row['type'] )

row = trial_df_en.iloc[5]
print("Source:", row['source'])
print("hyp1:", row['hyp1'])
print("hyp2:", row['hyp2'])
print("label:",row['label'] )
print("type:",row['type'] )

row = trial_df_en.iloc[4]
print("Source:", row['source'])
print("hyp1:", row['hyp1'])
print("hyp2:", row['hyp2'])
print("label:",row['label'] )
print("type:",row['type'] )

English
Source: Madam President, I am speaking on behalf of our colleague, Mr Francis Decourrière, who drafted one of the motions for a resolution.
hyp1: One of the motions for a resolution was drafted by Mr Francis Decourrière.
hyp2: One of the motions for a resolution was drafted by Mrs Francis Decourrière.
label: hyp2
type: gender
Source: This state of affairs has not changed in more than 100 years, but hopefully at some stage - and perhaps soon - change will come.
hyp1: There has been no change in the status quo in over 100 years, but there is hope that change will soon come. 
hyp2: The state of affairs is1-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-6556
label: hyp2
type: addition
Source: We need quite specific legisl

In [9]:
print('Swedish')
row = trial_df_se.iloc[0]  # Selecting the first row
print("Source:", row['source'])
print("hyp1:", row['hyp1'])
print("hyp2:", row['hyp2'])
print("label:",row['label'] )
print("type:",row['type'] )

row = trial_df_se.iloc[1]
print("Source:", row['source'])
print("hyp1:", row['hyp1'])
print("hyp2:", row['hyp2'])
print("label:",row['label'] )
print("type:",row['type'] )

row = trial_df_se.iloc[2]
print("Source:", row['source'])
print("hyp1:", row['hyp1'])
print("hyp2:", row['hyp2'])
print("label:",row['label'] )
print("type:",row['type'] )

Swedish
Source: Men intäkterna från mjukvarulicenser, ett mått som finansanalytiker följer noga, minskade med 21 procent till 107,6 miljoner dollar.
hyp1: Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.
hyp2: Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 42 procent till ett belopp av 107,6 miljoner dollar.
label: hyp2
type: number
Source: Hongkong-universitet samarbetar med universitet, företag och statliga sektorer i Kina för att samordna utbildningsprogram och forskningscentra för att främja högteknologisk forskning, kommersialisering och tekniköverföring.
hyp1: University of Hong Kong samarbetar med olika kinesiska universitet, företag och regeringsdepartement för att samordna utbildningsprogram och forskningscentra med syftet att främja avancerad teknologisk forskning, kommersialisering och tekniköverföring.
h

In [10]:
#load the test data for the generation step in both English and Swedish
data = load_dataset("Eloquent/HalluciGen-PG", "test_detection")

print(data.keys())

data_df_en = data['test_detection_english'].to_pandas()
data_df_se= data['test_detection_swedish'].to_pandas()


Generating test_detection_swedish split:   0%|          | 0/119 [00:00<?, ? examples/s]

Generating test_detection_english split:   0%|          | 0/119 [00:00<?, ? examples/s]

dict_keys(['test_detection_swedish', 'test_detection_english'])


In [11]:
data_df_en

,id,source,hyp1,hyp2
0,0,It has enabled us to support and encourage an ...,You can support and encourage an exchange of e...,We can support and encourage an exchange of ex...
1,1,"Therefore, I am calling for an increase in the...",I want to see a decrease in the premiums for a...,I want to see an increase in the premiums for ...
2,2,"In other words, a person may be prevented from...",When there is reason to fear that a violent ac...,When there is reason to fear that a violent ac...
3,3,None of my 34 amendments were adopted and my a...,Our arguments against the ridiculous administr...,My arguments against the ridiculous administra...
4,4,The final key feature of the report is its ori...,The final main feature of the report is its or...,The final features of the report are its orien...
...,...,...,...,...
114,114,I am referring to the current vote but in rela...,The vote on Amendment No 98 will take place be...,The vote on Amendment No 98 will take place af...
115,115,When I became a Member of the European Parliam...,I was struck by the abundance of knowledge whe...,I was struck by the lack of knowledge when I j...
116,116,"Mr President, let me express my thanks for the...","Mr President, I would like to extend a warm we...","Madam President, I would like to extend a warm..."
117,117,We can replace 16 out of 626 Members of the Eu...,"As voters we cannot decide on a new law, but w...","As voters decide on a new law, we can replace ..."


In [12]:
data_df_se

,id,source,hyp1,hyp2
0,0,Kvinnor kommer att möta högre bilförsäkringspr...,Det betyder att kvinnor kan förvänta sig att b...,Kvinnor kommer att få högre premier för bilför...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e...","Intäkterna från rörelsen var 1,45 miljarder do...","Intäkterna från rörelsen var 1,45 miljarder do..."
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ...",Mandela återvänder inte till sjukhuset med all...,Mandela återvänder till sjukhuset med allvarli...
3,3,Egypten fryser Muslimska brödraskapets tillgån...,Den egyptiska regeringen har fryst tillgångar ...,Egypten fryser tillgångarna för Muslimska bröd...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ...","Medicinen som vanligtvis kallas ""poly pill"" sä...","Medicinen som vanligtvis kallas ""poly pill"" sä..."
...,...,...,...,...
114,114,Texaner som dignar under skyhöga hemförsäkring...,Invånare i Texas som har belastats med orimlig...,Invånare i Texas som har belastats med orimlig...
115,115,Uppdaterad - Två explosioner nära mållinjen i ...,Uppdatering: Nära målet i Boston Marathon intr...,Två explosioner inträffade under loppet av Bos...
116,116,Anthony Weiner halkar ned till fjärde plats i ...,Anthony Weiner halkar till fjärde plats i ny o...,Enligt en ny opinionsundersökning från Quinnip...
117,117,Östtimor förbjuder kampsportsklubbar på grund ...,I Östtimor ska man förbjuda klubbar som utövar...,I Östtimor har man förbjudit klubbar som utöva...


In [13]:
# Replace "'" with '//"'
data_df_en['source'] = data_df_en['source'].str.replace("'", "\\'")

In [14]:
data_df_se['source'] = data_df_se['source'].str.replace("'", "\\'")

In [15]:
display(data_df_se)

,id,source,hyp1,hyp2
0,0,Kvinnor kommer att möta högre bilförsäkringspr...,Det betyder att kvinnor kan förvänta sig att b...,Kvinnor kommer att få högre premier för bilför...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e...","Intäkterna från rörelsen var 1,45 miljarder do...","Intäkterna från rörelsen var 1,45 miljarder do..."
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ...",Mandela återvänder inte till sjukhuset med all...,Mandela återvänder till sjukhuset med allvarli...
3,3,Egypten fryser Muslimska brödraskapets tillgån...,Den egyptiska regeringen har fryst tillgångar ...,Egypten fryser tillgångarna för Muslimska bröd...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ...","Medicinen som vanligtvis kallas ""poly pill"" sä...","Medicinen som vanligtvis kallas ""poly pill"" sä..."
...,...,...,...,...
114,114,Texaner som dignar under skyhöga hemförsäkring...,Invånare i Texas som har belastats med orimlig...,Invånare i Texas som har belastats med orimlig...
115,115,Uppdaterad - Två explosioner nära mållinjen i ...,Uppdatering: Nära målet i Boston Marathon intr...,Två explosioner inträffade under loppet av Bos...
116,116,Anthony Weiner halkar ned till fjärde plats i ...,Anthony Weiner halkar till fjärde plats i ny o...,Enligt en ny opinionsundersökning från Quinnip...
117,117,Östtimor förbjuder kampsportsklubbar på grund ...,I Östtimor ska man förbjuda klubbar som utövar...,I Östtimor har man förbjudit klubbar som utöva...


In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
!pip install -U "transformers==4.38.1" --upgrade
!pip install accelerate

# Load the model

In [16]:
#load model Llama3
from transformers import AutoTokenizer, pipeline
import torch

model = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Prompt Version 1

In [24]:
def detect_hallu(source, hyp1, hyp2):
   # Generate the prompt

  answer_format = {"label": " "}

  user_prompt = f'''
    You are a researcher investigating a new phenomenon.
    You have gathered data (source) and formulated two competing hypotheses (hyp1 and hyp2) to explain it.
    Identify the hypothesis that contradicts the information provided in the given source.
    Provide the result in the following format: {answer_format}.

    source: {source}
    hyp1: {hyp1}
    hyp2: {hyp2}

    Result:
  '''

  messages = [
    {"role": "system", "content": "You are a hallucination detector. Your task is to detect which hypothesis is a hallucination."},
    {"role": "user", "content": user_prompt},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=150,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
  print("Label: ", outputs[0]["generated_text"][len(prompt):])
  hallu_label = outputs[0]["generated_text"][len(prompt):]

  return hallu_label

In [ ]:
def detect_hallu_se(source, hyp1, hyp2):
   # Generate the prompt

    answer_format = {"label": ""}

    user_prompt = f'''
    Givet en ”src” och två hypoteser ”hyp1” och ”hyp2” är din uppgift att upptäcka vilken av de två hypoteserna (”label”) som inte stöds av källan.
    Ge resultatet i följande format: {answer_format}.

    Src: {source}
    hyp1 : {hyp1}
    hyp2 : {hyp2}

    Resultat:
    '''

    messages = [
      {"role": "system", "content": "Du är en hallucinationsdetektor. Din uppgift är att upptäcka vilken hypotes som är en hallucination."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    print("Label: ", outputs[0]["generated_text"][len(prompt):])
    hallu_label = outputs[0]["generated_text"][len(prompt):]


    return hallu_label

In [ ]:
def detect_hallu_2(source, hyp1, hyp2):

    # Generate the prompt

    answer_format = {"label": ""}
    user_prompt =  f'''
    As an AI model, your task is to determine which of the two hypotheses, "hyp1" and "hyp2," is not supported by the given source, labeled as either "hyp1" or "hyp2."

    Now, with the provided "src" and two hypotheses "hyp1" and "hyp2," identify the hypothesis that is not supported by the source.

    Src: {source}
    hyp1: {hyp1}
    hyp2: {hyp2}

    DO NOT GENERATE CODE AND ONLY ANSWER WITH THE GIVEN FORMAT USING THE FOLLOWING FORMAT: {answer_format}.



    Example 1:
    Src: The population has declined in some 210 of the 280 municipalities in Sweden, mainly in inland central and northern Sweden.
    hyp1: In the majority of Sweden's 280 municipalities, the population has gone up.
    hyp2: In the majority of Sweden's 280 municipalities, the population has gone down.

    Result:
    {"label": "hyp1"}

    Explanation: Hyp1 suggests that the population has increased in most municipalities, which contradicts the information provided in the source. The source clearly states that the population has declined in some municipalities, not increased.
    '''

    messages = [
      {"role": "system", "content": "You are a hallucination detector. Your task is to detect which hypothesis is a hallucination."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    print("Label: ", outputs[0]["generated_text"][len(prompt):])
    hallu_label = outputs[0]["generated_text"][len(prompt):]


    return hallu_label

In [18]:
import json
import re

def extract_json_data(result_text):

    print(result_text)
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    print("result_text:",result_text)
    return result_text

In [19]:
source = "My mom's food is better than my sister's."
hyp1= "My mom's food is amazing"
hyp2="My sister's food is better than my mom's"
result = detect_hallu(source,hyp1,hyp2)
print()
print(result)
result = extract_json_data(result)
print()

print("result:",result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}

{'label': 'hyp2'}
{'label': 'hyp2'}
result_text: {'label': 'hyp2'}

result: {'label': 'hyp2'}


## Generate dataset

In [20]:
import os
import numpy as np
import pandas as pd

#save results
def save_results(detect_hallu, df, csv_filename):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = detect_hallu(source, hyp1, hyp2)

                print(result)
                extracted_hallu = extract_json_data(result)

                hallu_label = extracted_hallu["label"]


                # Update DataFrame with new values
                chunk.at[index, "prediction"] = hallu_label
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows

# Evaluate prompts on the trial dataset

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_llama3/trial/results_llama3_prompt_narjes_en_v1_trial.csv"

save_results(detect_hallu, trial_df_en, csv_filename)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp1'}
{'label': 'hyp1'}
{'label': 'hyp1'}
result_text: {'label': 'hyp1'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp1'}
{'label': 'hyp1'}
{'label': 'hyp1'}
result_text: {'label': 'hyp1'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}

Hypothesis 2 contradicts the information provided in the source, as the source mentions a dual ceiling, which implies that there are two options: a 20% rate of taxation on interest payments or the supply of information. Hypothesis 2 only mentions one option, which is a 20% rate of taxation on interest payments.
{'label': 'hyp2'}

Hypothesis 2 contradicts the information provided in the source, as the source mentions a dual ceiling, which implies that there are two options: a 20% rate of taxation on interest payments or the supply of information. Hypothesis 2 only mentions one option, which is a 20% rate of taxation on interest payments.
{'label': 'hyp2'}

Hypothesis 2 contradicts the information provided in the source, as the source mentions a dual ceiling, which implies that there are two options: a 20% rate of taxation on interest payments or the supply of information. Hypothesis 2 only mentions one option, which is a 20% rate of taxation on interest paymen

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}

Hypothesis 1 (hyp1) is supported by the source, as it states that there is no other way to clean up the beaches on the North Sea and Baltic coasts, which matches the information provided.

Hypothesis 2 (hyp2) contradicts the source, as it mentions the Bothnian Bay coastline instead of the Baltic coasts, which is not mentioned in the source.
{'label': 'hyp2'}

Hypothesis 1 (hyp1) is supported by the source, as it states that there is no other way to clean up the beaches on the North Sea and Baltic coasts, which matches the information provided.

Hypothesis 2 (hyp2) contradicts the source, as it mentions the Bothnian Bay coastline instead of the Baltic coasts, which is not mentioned in the source.
{'label': 'hyp2'}

Hypothesis 1 (hyp1) is supported by the source, as it states that there is no other way to clean up the beaches on the North Sea and Baltic coasts, which matches the information provided.

Hypothesis 2 (hyp2) contradicts the source, as it mentions t

[]

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_llama3/trial/results_llama3_prompt_narjes_en_v1_trial.csv")

display(df)

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['prediction'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_llama3/trial/classification_report_llama3_prompt_narjes_en_v1_trial.txt", "w") as text_file:
    text_file.write(report)


,id,source,type,hyp1,hyp2,label,prediction
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1,hyp1
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2,hyp2
5,5,This state of affairs has not changed in more ...,addition,There has been no change in the status quo in ...,The state of affairs is1-65561-65561-65561-655...,hyp2,hyp2
6,6,"Madam President, I am speaking on behalf of ou...",gender,One of the motions for a resolution was drafte...,One of the motions for a resolution was drafte...,hyp2,hyp2
7,7,Amendment No 1 in the French version deletes i...,natural,The French version excludes the expression'pol...,The French version excludes the expression 'po...,hyp1,hyp2
8,8,The European Commission proposes that this inf...,number,The EU wants this information to enter into fo...,The EU wants this information to enter into fo...,hyp2,hyp2
9,9,"Mr President, I did prepare a speech but I hav...",pronoun,Many of the points they were going to make in ...,Many of the points I was going to make in my s...,hyp1,hyp2


              precision    recall  f1-score   support

        hyp1       1.00      0.67      0.80         9
        hyp2       0.70      1.00      0.82         7

    accuracy                           0.81        16
   macro avg       0.85      0.83      0.81        16
weighted avg       0.87      0.81      0.81        16



326

In [ ]:
#csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_llama3/trial/results_llama3_en_v2_trial.csv"


#save_results(detect_hallu_2, trial_df_en, csv_filename)

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_llama3/trial/results_llama3_prompt_narjes_en_v2_trial.csv")

display(df)

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['prediction'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_llama3/trial/classification_report_llama3_prompt_narjes_en_v2_trial.txt", "w") as text_file:
    text_file.write(report)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_llama3/trial/results_llama3_prompt_narjes_se_v1_trial.csv"


save_results(detect_hallu, trial_df_se, csv_filename)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}

Hypothesis 2 contradicts the information provided in the source, which states that the revenue from software licenses decreased by 21%, not 42%.
{'label': 'hyp2'}

Hypothesis 2 contradicts the information provided in the source, which states that the revenue from software licenses decreased by 21%, not 42%.
{'label': 'hyp2'}

Hypothesis 2 contradicts the information provided in the source, which states that the revenue from software licenses decreased by 21%, not 42%.
result_text: {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2 is a hallucination'}

The source text does not mention "regeringsdepartement" (government department) which is present in hyp2 but not in hyp1 or the source text. Therefore, hyp2 contradicts the information provided in the source and is considered a hallucination.
{'label': 'hyp2 is a hallucination'}

The source text does not mention "regeringsdepartement" (government department) which is present in hyp2 but not in hyp1 or the source text. Therefore, hyp2 contradicts the information provided in the source and is considered a hallucination.
{'label': 'hyp2 is a hallucination'}

The source text does not mention "regeringsdepartement" (government department) which is present in hyp2 but not in hyp1 or the source text. Therefore, hyp2 contradicts the information provided in the source and is considered a hallucination.
result_text: {'label': 'hyp2 is a hallucination'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}

Hypothesis 2 states that the Iranian government declared that the country's nuclear energy program "used to be" peaceful, which contradicts the information provided in the source, which states that the Iranian government declared that the country's nuclear energy program "will remain" peaceful.
{'label': 'hyp2'}

Hypothesis 2 states that the Iranian government declared that the country's nuclear energy program "used to be" peaceful, which contradicts the information provided in the source, which states that the Iranian government declared that the country's nuclear energy program "will remain" peaceful.
{'label': 'hyp2'}

Hypothesis 2 states that the Iranian government declared that the country's nuclear energy program "used to be" peaceful, which contradicts the information provided in the source, which states that the Iranian government declared that the country's nuclear energy program "will remain" peaceful.
result_text: {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}

Hypothesis 2 contradicts the information provided in the source, as it states that the speaker was employed by a company that the listener hired to erase their own memory, whereas the source states that the listener hired a company to erase the speaker's memory.
{'label': 'hyp2'}

Hypothesis 2 contradicts the information provided in the source, as it states that the speaker was employed by a company that the listener hired to erase their own memory, whereas the source states that the listener hired a company to erase the speaker's memory.
{'label': 'hyp2'}

Hypothesis 2 contradicts the information provided in the source, as it states that the speaker was employed by a company that the listener hired to erase their own memory, whereas the source states that the listener hired a company to erase the speaker's memory.
result_text: {'label': 'hyp2'}
Label:  {'label': 'hyp2'}

Hypothesis 2 contradicts the information provided in the source, as it states that Abu B

[]

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_llama3/trial/results_llama3_prompt_narjes_se_v1_trial.csv")

display(df)

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['prediction'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_llama3/trial/classification_report_llama3_prompt_narjes_se_v1_trial.txt", "w") as text_file:
    text_file.write(report)

,id,source,type,hyp1,hyp2,label,prediction
0,0,"Men intäkterna från mjukvarulicenser, ett mått...",number,"Intäkter från programvarulicenser, en metrik s...","Intäkter från programvarulicenser, en metrik s...",hyp2,hyp2
1,1,Hongkong-universitet samarbetar med universite...,natural,University of Hong Kong samarbetar med olika k...,University of Hong Kong samarbetar med olika k...,hyp1,hyp2 is a hallucination
2,2,Län med befolkningsminskning kommer att vara V...,named entity,"Vermillion, Posey och Madison är län som komme...","Vermillion, Posey och Marion är län som kommer...",hyp2,hyp2
3,3,Google presenterar en prototyp för en självkör...,addition,En prototyp för en självkörande bil presentera...,Enligt ett blogginlägg från Google har företag...,hyp2,hyp2
4,4,Lagförslaget säger att en kvinna som genomgår ...,negation,Förslaget stadgar att en kvinna som genomgår e...,Förslaget stadgar att en kvinna som genomgår e...,hyp1,hyp1
5,5,Spaniens prinsessa vittnar i historisk bedräge...,gender,Spanska prinsessan blir en del av en utredning...,Spanska prinsen blir en del av en utredning om...,hyp2,hyp2
6,6,Mannen använder en slägga för att bryta betong...,antonym,Mannen använder en slägga för att krossa beton...,Mannen använder en slägga för att krossa beton...,hyp2,hyp2
7,7,Nordkorea carnar utlänningar i söder och ger r...,negation,Nordkorea varnar inte utländska medborgare i s...,Nordkorea varnar utländska medborgare i södra ...,hyp1,hyp2
8,8,Beväpnad man bland 7 döda efter lägenhetsskjut...,addition,En beväpnad man var bland de sju döda efter en...,En man med ett skjutvapen har dödat minst sju ...,hyp2,hyp2
9,9,Irans kärnvapenförhandlingar går in på tredje ...,number,Diskussioner om irans kärnvapenprogram har nåt...,Diskussioner om irans kärnvapenprogram har nåt...,hyp2,hyp2


                         precision    recall  f1-score   support

                   hyp1       0.67      0.22      0.33         9
                   hyp2       0.62      0.91      0.74        11
hyp2 is a hallucination       0.00      0.00      0.00         0

               accuracy                           0.60        20
              macro avg       0.43      0.38      0.36        20
           weighted avg       0.64      0.60      0.56        20



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


457

# Generate predictions on the test data

## English

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_llama3/results_llama3_prompt_narjes_en_v1.csv"

save_results(detect_hallu, data_df_en, csv_filename)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}

Hypothesis 2 ("We can support...") contradicts the information provided in the source, which states "It has enabled us to support...". The use of "It has enabled" implies a past action or state, whereas "We can support" implies a future or present possibility, which is not supported by the source.
{'label': 'hyp2'}

Hypothesis 2 ("We can support...") contradicts the information provided in the source, which states "It has enabled us to support...". The use of "It has enabled" implies a past action or state, whereas "We can support" implies a future or present possibility, which is not supported by the source.
{'label': 'hyp2'}

Hypothesis 2 ("We can support...") contradicts the information provided in the source, which states "It has enabled us to support...". The use of "It has enabled" implies a past action or state, whereas "We can support" implies a future or present possibility, which is not supported by the source.
result_text: {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp1'}
{'label': 'hyp1'}
{'label': 'hyp1'}
result_text: {'label': 'hyp1'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}
{'label': 'hyp2'}
{'label': 'hyp2'}
result_text: {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}

Hypothesis 2 contradicts the information provided in the source, which states that "we can replace 16 out of 626 Members of the European Parliament, but we can never again as voters decide to bring in a new law." This implies that voters cannot decide on a new law, which is the opposite of what hypothesis 2 claims.
{'label': 'hyp2'}

Hypothesis 2 contradicts the information provided in the source, which states that "we can replace 16 out of 626 Members of the European Parliament, but we can never again as voters decide to bring in a new law." This implies that voters cannot decide on a new law, which is the opposite of what hypothesis 2 claims.
{'label': 'hyp2'}

Hypothesis 2 contradicts the information provided in the source, which states that "we can replace 16 out of 626 Members of the European Parliament, but we can never again as voters decide to bring in a new law." This implies that voters cannot decide on a new law, which is the opposite of what hypot

[]

In [ ]:
# save a csv file in the task format

import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_llama3/results_llama3_prompt_narjes_en_v1.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'prediction']]
df = df.rename(columns={'prediction': 'label'})

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_llama3_prompt_narjes_en_v1_detection.csv', index=False)

## Swedish

In [25]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_llama3/results_llama3_prompt_narjes_se_v2.csv"

save_results(detect_hallu, data_df_se, csv_filename)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}
{'label': 'hyp2'}
{'label': 'hyp2'}
result_text: {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp1'}
{'label': 'hyp1'}
{'label': 'hyp1'}
result_text: {'label': 'hyp1'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}
{'label': 'hyp2'}
{'label': 'hyp2'}
result_text: {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}
{'label': 'hyp2'}
{'label': 'hyp2'}
result_text: {'label': 'hyp2'}
Label:  {'label': 'hyp2'}
{'label': 'hyp2'}
{'label': 'hyp2'}
result_text: {'label': 'hyp2'}


[]

In [26]:
# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_llama3/results_llama3_prompt_narjes_se_v2.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'prediction']]
df = df.rename(columns={'prediction': 'label'})

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_llama3_prompt_narjes_se_v2_detection.csv', index=False)

In [ ]:
display(df)

,id,label
0,0,hyp2
1,1,hyp1
2,2,hyp2
3,3,hyp2
4,4,hyp2
...,...,...
114,114,hyp2
115,115,hyp1
116,116,hyp2
117,117,hyp2
